# Lights Out: Forecasting Residential Impact of Power Outages
(Predict the percentage of residential customers (RES.CUST.PCT) affected by a major power outage)

**Name(s)**: Luke Lin, Andrew Yin

**Website Link**: (your website link)

## Imports

In [66]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

## Framing the Problem

In this project, we are tackling a regression problem. Our goal is to predict the percentage of residential customers (RES.CUST.PCT) affected by a major power outage. We chose this as our response variable because it provides a quantifiable measure of the impact of power outages on residential customers.

(CHANGE LINE AS SOON AS ANDREW PUSHES HIS CODE)
The features we are using to train our model include U.S._STATE and OUTAGE.DURATION. These features were chosen because they are known at the “time of prediction” and are expected to have a significant influence on the percentage of residential customers affected by an outage.

## Model Evaluation
To evaluate our model, we are using the Root Mean Squared Error (RMSE). This metric was chosen because it is suitable for regression problems and it penalizes large errors more due to the squaring of the residuals. This makes it a good choice for our problem where we want to minimize the difference between the actual and predicted percentages of residential customers affected by an outage.

## Feature Selection
Our target variable is RES.CUST.PCT, which represents the percentage of residential customers affected by a major power outage. To predict this, we’ve selected two features: U.S._STATE and OUTAGE.DURATION.

- U.S._STATE: This is a categorical variable representing the U.S. state where the outage occurred. The rationale behind including this feature is that the impact of an outage can vary by location due to factors such as infrastructure, population density, and local policies. By including the state as a feature, our model can learn these regional differences.
- OUTAGE.DURATION: This is a numerical variable representing the duration of the power outage. It’s reasonable to assume that longer outages will affect more customers, making this a potentially powerful feature for our prediction problem.
- ADD ANDREW"S FEATURES LATER

## Cleaning the Data
In this section, we’ve performed a series of data cleaning steps on the outage DataFrame, which was read from an Excel file named “outage.xlsx”. This process was a direct replication of the cleaning procedure implemented in Project 3, ensuring consistency and reliability in our data preparation phase.

Initially, we removed informational rows and columns that contained only null values from the DataFrame. We then set the column names based on the first row of the DataFrame for better readability and understanding of the data.

Next, we dropped rows related to units and variables that were not necessary for our analysis. We also dropped the “variables” column as it was not needed.

One of the crucial steps in this process was the creation of new datetime columns, OUTAGE.START and OUTAGE.RESTORATION. These were formed by combining the respective date and time columns, providing us with precise timestamps of when the outage started and ended. After creating these new columns, the original date and time columns were dropped to avoid redundancy.

Lastly, we replaced the “NA” entries with NaN for missing values.

After these cleaning steps, we were left with a cleaned DataFrame, outage_cleaned, ready for further analysis or modeling.

In [67]:
# Read the Excel file into a pandas DataFrame 
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")

# Drop informational rows
outage_cleaned = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names based on the first row
outage_cleaned.columns = outage_cleaned.iloc[0]

# Drop rows related to units and variables
outage_cleaned = outage_cleaned.drop([4, 5])
outage_cleaned = outage_cleaned.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a new datetime column
outage_cleaned['OUTAGE.START'] = pd.to_datetime(outage_cleaned['OUTAGE.START.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a new datetime column
outage_cleaned['OUTAGE.RESTORATION'] = pd.to_datetime(outage_cleaned['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Replace "NA" with NaN for missing values
outage_cleaned.replace("NA", np.nan, inplace=True)

print(outage_cleaned.shape)

(1534, 54)


## Data Preprocessing

In this section, we perform several preprocessing steps on the outage_cleaned DataFrame to prepare it for further analysis or modeling. The processed data is stored in a new DataFrame called processed_outage.

### 1. Handling Missing Values
We first handle missing values in the dataset. For numerical columns, we fill missing values with the mean of the respective column. For categorical columns, we fill missing values with the mode (most frequent value) of the respective column.

## 2. Converting Date/Time Columns
We convert the OUTAGE.START and OUTAGE.RESTORATION columns from string format to datetime format using pandas’ to_datetime function. This allows us to perform datetime-specific operations on these columns.

## 3. Encoding Categorical Variables
Finally, we encode the categorical variables in the dataset. This is necessary because many machine learning algorithms require the input data to be numerical. We use scikit-learn’s LabelEncoder to transform the categorical values into numerical labels. This also made it easier to visualize all of the 50 states in a 3d scattor plot instead of long strings that made the graph messy. 

This completes the data preprocessing section. The processed_outage DataFrame is now ready for further analysis or modeling.

In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

processed_outage = outage_cleaned.copy()

# Fill numerical columns with the mean
num_cols = processed_outage.select_dtypes(include=[np.number]).columns
processed_outage[num_cols] = processed_outage[num_cols].fillna(processed_outage[num_cols].mean())

# Fill categorical columns with the mode
cat_cols = processed_outage.select_dtypes(include=['object']).columns
processed_outage[cat_cols] = processed_outage[cat_cols].fillna(processed_outage[cat_cols].mode().iloc[0])

# 2. Convert Date/Time Columns
processed_outage['OUTAGE.START'] = pd.to_datetime(processed_outage['OUTAGE.START'])
processed_outage['OUTAGE.RESTORATION'] = pd.to_datetime(processed_outage['OUTAGE.RESTORATION'])

# Create a new feature for outage duration in minutes
processed_outage['OUTAGE.DURATION'] = (processed_outage['OUTAGE.RESTORATION'] - processed_outage['OUTAGE.START']).dt.total_seconds() / 60

# 3. Encode Categorical Variables
le = LabelEncoder()
for col in cat_cols:
    processed_outage[col] = le.fit_transform(processed_outage[col])

## Baseline Model Training and Evaluation
In this section, we train a linear regression model to predict the percentage of residential customers (RES.CUST.PCT) affected by a major power outage, using U.S._STATE and OUTAGE.DURATION as features.

## 1. Feature Selection
We first select our features and target from the processed_outage DataFrame:

## 2. Train-Test Split
We split our data into a training set and a test set, with 70% of the data used for training the model and 30% used for evaluating its performance.

## 3. Preprocessing
We define separate preprocessing steps for the numerical and categorical features. The numerical features are scaled using StandardScaler, and the categorical features are encoded using OneHotEncoder. These preprocessing steps are combined into a ColumnTransformer.

## 4. Pipeline Creation
We create a pipeline that first applies the preprocessing steps and then trains a LinearRegression model.

## 5. Model Training
We train the model using the training data.

## 6. Model Evaluation
Finally, we evaluate the model’s performance on the test set by predicting the target variable and calculating the root mean squared error (RMSE).

This completes the model training and evaluation section. The RMSE gives us a measure of how well our model is able to predict the percentage of residential customers affected by a major power outage.

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Select features and target
features = processed_outage[['U.S._STATE', 'OUTAGE.DURATION']]
target = processed_outage['RES.CUST.PCT']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Define preprocessing for numerical columns (scale them)
num_processor = Pipeline([
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical columns (encode them)
cat_processor = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_processor, ['OUTAGE.DURATION']),
        ('cat', cat_processor, ['U.S._STATE'])
    ])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

Test RMSE: 21.381649961448115


## Hyperparameter Tuning and Model Evaluation
In this section, we perform hyperparameter tuning using GridSearchCV and evaluate the performance of the best model.

## 1. Define the Parameter Grid
We first define a parameter grid for the LinearRegression model. In this case, we want to tune the fit_intercept parameter, which indicates whether or not to calculate the intercept for this model:

## 2. Create the GridSearchCV Object
We then create a GridSearchCV object with the pipeline as the estimator, the parameter grid, and 5-fold cross-validation. The scoring metric is the negative mean squared error because GridSearchCV tries to maximize the score, so to minimize the mean squared error, we need to take its negative:

## 3. Fit the GridSearchCV Object to the Data
We fit the GridSearchCV object to the training data, which performs cross-validation and hyperparameter tuning:

## 4. Evaluate the Best Model on the Test Set
Finally, we evaluate the performance of the best model on the test set by predicting the target variable and calculating the RMSE:

This completes the hyperparameter tuning and model evaluation section. The RMSE gives us a measure of how well our model is able to predict the percentage of residential customers affected by a major power outage. As we see, including or excluding the intercept did not affect our performance as much. 

In [70]:
from sklearn.model_selection import GridSearchCV

features = processed_outage[['U.S._STATE', 'OUTAGE.DURATION']]
target = processed_outage['RES.CUST.PCT']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Define preprocessing for numerical columns (scale them)
num_processor = Pipeline([
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical columns (encode them)
cat_processor = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_processor, ['OUTAGE.DURATION']),
        ('cat', cat_processor, ['U.S._STATE'])
    ])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Define the parameter grid
param_grid = {
    'regressor__fit_intercept': [True, False]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit it to the data and find the best parameters
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding RMSE on the training set
print('Best parameters:', grid_search.best_params_)
print('Best RMSE on training set:', np.sqrt(-grid_search.best_score_))

# Evaluate the best model on the test set
y_pred = grid_search.predict(X_test)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

Best parameters: {'regressor__fit_intercept': False}
Best RMSE on training set: 23.268304102560073
Test RMSE: 21.381654897236032


## Model Predictions and Visualization
In this section, we use the best model from the grid search to make predictions and visualize the results.

## 1. Predicted vs Actual Values
We first use the best model to make predictions on the test set. We then create a scatter plot of the predicted values versus the actual values. This plot helps us understand how well our model’s predictions align with the actual values. Each point represents an instance in the test set. The closer a point is to the line of perfect predictions (where the predicted value is equal to the actual value), the more accurate the prediction.

As we can see, simply by using US States and OUTAGE DURATION, we could closely approximate the number of residential customers affected. 

## 2. 3D Scatter Plot of Predictions
Next, we use the best model to make predictions on the entire dataset. We then create a 3D scatter plot with OUTAGE.DURATION and POPULATION as the x and y axes, and the predicted RES.CUST.PCT as the z-axis. The points are colored by U.S._STATE. This plot helps us visualize the relationship between the features and the predicted target variable in three dimensions. It can be especially useful when dealing with multiple features, as it allows us to explore potential interactions and patterns in the data. 



In [71]:
import plotly.graph_objects as go
import plotly.express as px

# Use the best model to make predictions on the test set
y_pred = grid_search.predict(X_test)

# Create a scatter plot of predicted vs actual values
fig = go.Figure(data=go.Scatter(x=y_test, y=y_pred, mode='markers'))

# Add a line for perfect predictions
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Perfect predictions'))

# Add title and labels
fig.update_layout(title='Predicted vs Actual Values',
                  xaxis_title='Actual Values',
                  yaxis_title='Predicted Values')

# Show the plot
fig.show()

# Use the best model to make predictions
ploted_outage = processed_outage.copy()
ploted_outage['PREDICTED'] = grid_search.predict(ploted_outage[['U.S._STATE', 'OUTAGE.DURATION']])

# Create a 3D scatter plot
fig = px.scatter_3d(ploted_outage, x='OUTAGE.DURATION', y='POPULATION', z='PREDICTED', color='U.S._STATE')
fig.show()



### Final Model

## Feature Selection
The model considers the following features:
- `U.S._STATE`: The state where the outage occurred.
- `OUTAGE.DURATION`: Duration of the outage event.
- `DEMAND.LOSS.MW`: The amount of peak demand lost during an outage.
- `POPULATION`: The population in the area of the outage.
- `CUSTOMERS.AFFECTED`: Number of customers affected by the outage.

## Data Splitting
The dataset is divided into training and testing sets using a 70-30 split, ensuring a random selection of data points for each set (`random_state=42` for reproducibility).

## Preprocessing Steps
The preprocessing involves:
- **StandardScaler**: Applied to `OUTAGE.DURATION` and `DEMAND.LOSS.MW` to normalize these features.
- **QuantileTransformer**: Transforms `POPULATION` and `CUSTOMERS.AFFECTED` to follow a normal distribution, reducing the impact of outliers.
- **OneHotEncoder**: Encodes the categorical feature `U.S._STATE`.

## Model and Pipeline
- A RandomForestRegressor is chosen for its ability to handle non-linear relationships and its robustness against overfitting.
- A pipeline is created combining the preprocessing steps and the model, ensuring a streamlined process from data transformation to model training.

## Hyperparameter Tuning
- **GridSearchCV** is used for hyperparameter tuning, focusing on:
  - `n_estimators`: Number of trees in the forest.
  - `max_depth`: Maximum depth of the trees.
  - `min_samples_split`: Minimum number of samples required to split a node.
- The grid search is performed with 5-fold cross-validation.

## Model Training and Evaluation
- The model is trained using the training dataset.
- The best model parameters are determined and printed.
- Finally, the model's performance is evaluated on the test dataset using the RMSE (Root Mean Squared Error) metric, providing a measure of the model's prediction accuracy.

In [91]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Select features and target
features = processed_outage[['U.S._STATE', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'POPULATION', 'CUSTOMERS.AFFECTED']]
target = processed_outage['RES.CUST.PCT']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Preprocessing for numerical columns
num_processor = Pipeline([
    ('scaler', StandardScaler())
])

qnt_processor = Pipeline([
    ('quantile', QuantileTransformer(n_quantiles=400, output_distribution='normal'))
])

# Preprocessing for categorical columns
cat_processor = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_processor, ['OUTAGE.DURATION', 'DEMAND.LOSS.MW']),
        ('qnt', qnt_processor, ['POPULATION', 'CUSTOMERS.AFFECTED']),
        ('cat', cat_processor, ['U.S._STATE'])
    ])

# Define the model
model = RandomForestRegressor()

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Define the grid search
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20],
    'model__min_samples_split': [2, 5]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# Fit the model
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

print('Best params:', grid_search.best_params_)

# Evaluate the model
y_pred = best_model.predict(X_test)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))


Best params: {'model__max_depth': 20, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Test RMSE: 8.63108649674028


## Predictions
- The model predictions (`y_pred`) are generated using the `predict` method of the `grid_search` object, which contains the trained model. These predictions are based on the `X_test` dataset.

## Creating the Scatter Plot
- A scatter plot is created using `go.Figure` and adding a scatter trace with `go.Scatter`. The plot has actual values (`y_test`) on the x-axis and predicted values (`y_pred`) on the y-axis.
- Each point in the scatter plot represents an individual observation from the test set, with its actual and predicted value.

## Adding a Line for Perfect Predictions
- A line representing perfect predictions is added to the plot. This line is where the predicted values exactly match the actual values. It's created using another `go.Scatter` trace where both x and y are set to `y_test`.
- The addition of this line provides a reference for evaluating the accuracy of the model's predictions. Points lying on or close to this line indicate accurate predictions.

In [92]:
import plotly.graph_objects as go
import plotly.express as px

# Use the best model to make predictions on the test set
y_pred = grid_search.predict(X_test)

# Create a scatter plot of predicted vs actual values
fig = go.Figure(data=go.Scatter(x=y_test, y=y_pred, mode='markers'))

# Add a line for perfect predictions
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Perfect predictions'))

# Add title and labels
fig.update_layout(title='Predicted vs Actual Values',
                  xaxis_title='Actual Values',
                  yaxis_title='Predicted Values')

# Show the plot
fig.show()
fig.write_html("Assets/new_regression.html")


### Fairness Analysis

## Grouping Data
- The code defines two groups: 'Northern Half' and 'Everything Else' based on the 'CLIMATE.REGION' values. The 'Northern Half' group includes regions such as Northeast, Northwest, East North Central, and West North Central. The 'Everything Else' group comprises all other regions.
- It calculates the RMSE for both groups based on the model's predictions (`y_pred`), resulting in `rmse_X_original` and `rmse_Y_original`.

## Permutation Test
- A permutation test is conducted to assess whether the observed difference in RMSE between the two groups is statistically significant. The process involves the following steps:
    1. For a specified number of permutations (`n_permutations`), the 'CLIMATE.REGION' labels in `X_test` are shuffled randomly.
    2. RMSE is recalculated for the shuffled groups, resulting in `rmse_X_perm` and `rmse_Y_perm`.
    3. The absolute difference between `rmse_X_perm` and `rmse_Y_perm` is computed and stored in `perm_diffs`.
- After running all permutations, the code calculates a p-value by determining how often the permuted differences are greater than or equal to the observed difference (`observed_diff`).
- The p-value represents the probability of observing a difference in RMSE as extreme as the one observed if the 'CLIMATE.REGION' labels have no effect on the model's performance.


In [88]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Assuming processed_outage is your dataset and it contains a 'CLIMATE.REGION' column

# Adding a 'CLIMATE.REGION' column to X_test for demonstration purposes
# In practice, this information should come from your actual test dataset

X_test['CLIMATE.REGION'] = outage_cleaned['CLIMATE.REGION']

# Define the Northern Half and Everything Else
northern_half = ['Northeast', 'Northwest', 'East North Central', 'West North Central']
group_X_indices = X_test['CLIMATE.REGION'].isin(northern_half)
group_Y_indices = ~X_test['CLIMATE.REGION'].isin(northern_half)

# Calculate RMSE for each group
rmse_X_original = np.sqrt(mean_squared_error(y_test[group_X_indices], y_pred[group_X_indices]))
rmse_Y_original = np.sqrt(mean_squared_error(y_test[group_Y_indices], y_pred[group_Y_indices]))
observed_diff = abs(rmse_X_original - rmse_Y_original)

# Permutation test
n_permutations = 1000
perm_diffs = []

for _ in range(n_permutations):
    # Shuffle the 'CLIMATE.REGION' labels
    shuffled_regions = np.random.permutation(X_test['CLIMATE.REGION'])
    shuffled_regions = pd.Series(shuffled_regions, index=X_test.index)  # Convert to pandas Series

    # Recalculate RMSE for the shuffled groups
    rmse_X_perm = np.sqrt(mean_squared_error(y_test[shuffled_regions.isin(northern_half)], y_pred[shuffled_regions.isin(northern_half)]))
    rmse_Y_perm = np.sqrt(mean_squared_error(y_test[~shuffled_regions.isin(northern_half)], y_pred[~shuffled_regions.isin(northern_half)]))

    perm_diffs.append(abs(rmse_X_perm - rmse_Y_perm))

# Calculate p-value
p_value = np.mean([diff >= observed_diff for diff in perm_diffs])

print('P-value:', p_value)

P-value: 0.054
